**Mount to your drive**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

PATH = "/content/gdrive/MyDrive"
%cd $PATH

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive


**Clone Git**

In [ ]:
#!git clone https://github.com/madilabcode/scNET
%cd "./scNET"

/content/gdrive/MyDrive/scNET


**Download example data - and h5ad object**

In [ ]:
import gdown
download_url = f'https://drive.google.com/uc?id=1YxQL1_BcbeAqVS7KhqzFzIJIyZjtLCcG'
output_path = './Data/example.h5ad'
gdown.download(download_url, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1YxQL1_BcbeAqVS7KhqzFzIJIyZjtLCcG
From (redirected): https://drive.google.com/uc?id=1YxQL1_BcbeAqVS7KhqzFzIJIyZjtLCcG&confirm=t&uuid=19763705-368a-437c-b0fe-f41dc2af6796
To: /content/gdrive/MyDrive/scNET/Data/example.h5ad
100%|██████████| 967M/967M [00:09<00:00, 98.0MB/s]


'./Data/example.h5ad'

**Downlaod Packages**

In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

#!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install scanpy
!pip install mygene

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.9 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=79ae363742a2c397f8129c459881e1aa70b70f23121646f9f73953622b864b93
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8027 sha256=1f89e4f15cf17f1c7e2cd3fa70b34850138b5125a6d34bb391d36e31e87d2b8d
  Stored in di

**Imports**

In [ ]:
import os
import torch
from main import main
import warnings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
torch.manual_seed(42)

**Train the model - the model, embeddings and KNN will be saved**


In [ ]:
main(path = r"./Data/bcells.h5ad", pre_processing_flag=False, train_all = True, human_flag=True, random_noise=False ,number_of_batches=5, cell_flag = False, model_name = "")


446
tensor(33.0293, device='cuda:0', grad_fn=<AddBackward0>)
row loss:129.7225799560547, col loss:9.703920364379883
Epoch: 000, AUC: 0.5665, AP: 0.5701
best result of model with auc: 0.5665101610131056
tensor(3.1002, device='cuda:0', grad_fn=<AddBackward0>)
row loss:5.769832611083984, col loss:9.182676315307617
Epoch: 010, AUC: 0.8152, AP: 0.8443
best result of model with auc: 0.8152454850633936


KeyboardInterrupt: 